In [ ]:
import json

In [ ]:
with open('primary_dataset_new.json') as f:
    data = json.loads(f.read())

In [ ]:
data = data['data']

In [ ]:
def get_wikipedia_url_from_wikidata_id(wikidata_id, lang='en', debug=False):
    import requests
    from requests import utils

    url = (
        'https://www.wikidata.org/w/api.php'
        '?action=wbgetentities'
        '&props=sitelinks/urls'
        f'&ids={wikidata_id}'
        '&format=json')
    json_response = requests.get(url).json()
    if debug: print(wikidata_id, url, json_response) 

    entities = json_response.get('entities')    
    if entities:
        entity = entities.get(wikidata_id)
        if entity:
            sitelinks = entity.get('sitelinks')
            if sitelinks:
                if lang:
                    # filter only the specified language
                    sitelink = sitelinks.get(f'{lang}wiki')
                    if sitelink:
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            return requests.utils.unquote(wiki_url)
                else:
                    # return all of the urls
                    wiki_urls = {}
                    for key, sitelink in sitelinks.items():
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            wiki_urls[key] = requests.utils.unquote(wiki_url)
                    return wiki_urls
    return None  

In [ ]:
data_new = []
done = 0
for ent in tqdm(data):
    ent['en_link'] = get_wikipedia_url_from_wikidata_id(ent['wd_id'])
    data_new.append(ent)
    done+=1

In [ ]:
print(done)
print(data_new[:5])

In [ ]:
with open('primary_dataset_new_withlinks.json',"w+") as f:
    json.dump(data_new , f)